# File untuk melakukan pengujian terhadap model yang sudah di deploy.

Melakukan import library yang dibutuhkan untuk keseluruhan proyek.

In [1]:
# Import library
import pandas as pd
import tensorflow as tf
import base64
import json
import requests

Melakukan load terhadap dataset dan menampilkan isinya.

In [2]:
# Load data
data = pd.read_csv('data/data.csv')
data

,Age,Gender,EducationLevel,ExperienceYears,PreviousCompanies,DistanceFromCompany,InterviewScore,SkillScore,PersonalityScore,RecruitmentStrategy,HiringDecision
0,26,1,2,0,3,26.783828,48,78,91,1,1
1,39,1,4,12,3,25.862694,35,68,80,2,1
2,48,0,2,3,2,9.920805,20,67,13,2,0
3,34,1,2,5,2,6.407751,36,27,70,3,0
4,30,0,1,6,1,43.105343,23,52,85,2,0
...,...,...,...,...,...,...,...,...,...,...,...
1495,48,0,2,3,4,9.183783,66,3,80,3,1
1496,27,1,2,10,3,14.847731,43,97,7,2,0
1497,24,1,1,1,2,4.289911,31,91,58,1,1
1498,48,0,2,4,4,36.299263,9,37,44,2,1


Mengambil bari pertama dari dataset dengan mengabaikan kolom target.

In [3]:
# Mengambil input
inputs = data.iloc[0, :10].to_dict()
inputs

{'Age': 26.0,
 'Gender': 1.0,
 'EducationLevel': 2.0,
 'ExperienceYears': 0.0,
 'PreviousCompanies': 3.0,
 'DistanceFromCompany': 26.78382755385365,
 'InterviewScore': 48.0,
 'SkillScore': 78.0,
 'PersonalityScore': 91.0,
 'RecruitmentStrategy': 1.0}

Mengubah data agar dapat diakses oleh model dan mengirimkan request predict ke endpoint.

In [4]:
# Fungsi untuk menyiapkan data input
def prepare_json(inputs: dict):

    # Mendefinisikan mapping dari kunci input ke jenis Fitur TensorFlow
    feature_mapping = {
        "Age": tf.train.Feature(int64_list=tf.train.Int64List(value=[int(inputs['Age'])])),
        "Gender": tf.train.Feature(int64_list=tf.train.Int64List(value=[int(inputs['Gender'])])),
        "EducationLevel": tf.train.Feature(int64_list=tf.train.Int64List(value=[int(inputs['EducationLevel'])])),
        "ExperienceYears": tf.train.Feature(int64_list=tf.train.Int64List(value=[int(inputs['ExperienceYears'])])),
        "PreviousCompanies": tf.train.Feature(int64_list=tf.train.Int64List(value=[int(inputs['PreviousCompanies'])])),
        "DistanceFromCompany": tf.train.Feature(float_list=tf.train.FloatList(value=[float(inputs['DistanceFromCompany'])])),
        "InterviewScore": tf.train.Feature(int64_list=tf.train.Int64List(value=[int(inputs['InterviewScore'])])),
        "SkillScore": tf.train.Feature(int64_list=tf.train.Int64List(value=[int(inputs['SkillScore'])])),
        "PersonalityScore": tf.train.Feature(int64_list=tf.train.Int64List(value=[int(inputs['PersonalityScore'])])),
        "RecruitmentStrategy": tf.train.Feature(int64_list=tf.train.Int64List(value=[int(inputs['RecruitmentStrategy'])])),
    }

    # Membuat TensorFlow Example
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_mapping)
    ).SerializeToString()

    # Melakukan encode Example ke base64
    result = {
        "examples": {
            "b64": base64.b64encode(example).decode()
        }
    }

    # Mengembalikan data dalam format JSON
    return json.dumps({
        "signature_name": "serving_default",
        "instances": [result],
    })


# Fungsi untuk memprediksi data
def predict(inputs):
    json_data = prepare_json(inputs)

    endpoint = "https://proyek-akhir-mlops-production.up.railway.app/v1/models/cc-model:predict"

    response = requests.post(endpoint, data=json_data)

    prediction = response.json().get("predictions")[0][0]
    if prediction:
        if prediction > 0.5:
            result = "Label 1 (Hired)"
        elif prediction < 0.5:
            result = "Label 0 (Not Hired)"
        else:
            result = "Unknown label"
    else:
        result = "Error: No predictions found in response."
    
    return result



Melakukan prediksi menggunakan data pada baris pertama. Output seharusnya mengembalikan 1 (Hired)

In [5]:
# Predict data
inputs = data.iloc[0, :10].to_dict()
print(predict(inputs))

Label 1 (Hired)


Melakukan prediksi menggunakan data pada baris keempat. Output seharusnya mengembalikan 0 (Not hired)

In [15]:
# Predict data
inputs = data.iloc[3, :10].to_dict()
print(predict(inputs))

Label 0 (Not Hired)


Lakukan prediksi pada data lainnya dengan mengubah nilai X dengan nomor baris yang diinginkan.

`data.iloc[X, :10].to_dict()`